In [ ]:
import numpy as np
from my_functions import *
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [ ]:
split_converter = lambda s: s.split()
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))
tile_name = lambda s: s[:8]

cat = pd.read_csv('csv/minijpas.MagAB_aper3_photoz_gaia_4.csv', sep=',', header=1,
    converters={2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags, 21: tile_name})

cat = cat[np.array([len(x) for x in cat['MAG_APER_3_0']]) != 0]

In [ ]:
def expfit(x, a, b, c):
    return a * np.exp(b * x + c)

In [ ]:
params_tiles = []
for tile in ['AEGIS001', 'AEGIS002', 'AEGIS003', 'AEGIS004']:
    err_fit_params = []

    mag = np.stack(
        cat['MAG_APER_3_0'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T
    mag_err = np.stack(
        cat['MAG_ERR_APER_3_0'][np.where(cat['name'] == tile)[0]].to_numpy()
    ).astype(float).T

    for nb_c in range(60):
        mask = (mag[nb_c] < 90) & (mag_err[nb_c] < 90)
        p0 = [1., 1., 1.]
        p, var = curve_fit(expfit, mag[nb_c, mask], mag_err[nb_c, mask], p0=p0)
        err_fit_params.append(p)

    params_tiles.append(err_fit_params)

In [ ]:
params_tiles = np.array(params_tiles)
params_tiles.shape

In [ ]:
params_tiles = np.median(params_tiles, axis=0)
params_tiles.shape

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

nb_c = 30
a = err_fit_params[nb_c][0]
b = err_fit_params[nb_c][1]
c = err_fit_params[nb_c][2]

mask = (mag[nb_c] < 90) & (mag_err[nb_c] < 90)

ax.scatter(mag[nb_c, mask], mag_err[nb_c, mask])

x_Arr = np.linspace(13, 28, 100)
ax.plot(x_Arr, expfit(x_Arr, a, b, c), zorder=99, c='red')

ax.set_ylim((0, 20))

plt.show()

In [ ]:
np.save('npy/err_fit_params_minijpas', np.array(err_fit_params))

In [ ]:
for i in range(len(err_fit_params)):
    a = err_fit_params[i][0]
    b = err_fit_params[i][1]
    c = err_fit_params[i][2]
    print(expfit(24, a, b, c))